# Setup

* First, we need to import the necessary libraries.
* Then we load the configuration file.
  * The config file contains information like the database config and the API key
* Then we setup the SQLite database

In [17]:
import requests
import time
import sqlite3
from datetime import datetime
from IPython.display import JSON
import json
import configparser as cp
import pandas as pd

In [18]:
"""
    Reads the config file and returns the config object
"""
config = cp.ConfigParser()
config.read('config.ini')

['config.ini']

## SQLite

I decided to user a SQLite database instead of simply keeping the data in memory or in a JSON file\nnot just because of the size of the dataset, but also because of the inherent structure of the data itself.

As it is to be expected that the the n to n relationship between the Repos and the Contributors will be queried a lot, so it makes more sense to have a database which can handle relational requests, instead of manually joining dataframes.

In [20]:
db = sqlite3.connect(config['DB']['NAME'])
   

db.execute("""
           CREATE TABLE IF NOT EXISTS users (
               id TEXT PRIMARY KEY,
               name TEXT NOT NULL,
               location TEXT,
               addedToDB datetime,
               createdAt datetime
            )
              """)

db.execute(""" 
           CREATE TABLE IF NOT EXISTS repos (
               id TEXT PRIMARY KEY,
               name TEXT NOT NULL,
               owner TEXT,
               url TEXT,
               stargazerCount INTEGER,
               watchers INTEGER,
               primaryLanguage text,
               isFork boolean,
               forkCount INTEGER,
               updatedAt datetime,
               createdAt datetime,
               addedToDB datetime,
               allCommits boolean,
               FOREIGN KEY (owner) REFERENCES users(id)
            )
           """)

db.execute("""
           CREATE TABLE IF NOT EXISTS commits (
               id TEXT PRIMARY KEY,
               repo TEXT NOT NULL,
               user TEXT,
               createdAt datetime,
               additions INTEGER,
               deletions INTEGER,
               addedToDB datetime,
               FOREIGN KEY (repo) REFERENCES repos(id),
               FOREIGN KEY (user) REFERENCES users(id)
            )
            """)

db.commit()

In [21]:
# Query to insert a repo into the database
INSERT_REPO = """
                        INSERT OR IGNORE INTO repos(
                            id,
                            name,
                            owner,
                            url,
                            stargazerCount,
                            watchers,
                            %s
                            isFork,
                            forkCount,
                            updatedAt,
                            createdAt,
                            addedToDB,
                            allCommits
                        )
                        VALUES("%s", "%s", "%s","%s",  %s,  %s, %s %s, %s, %s, %s, CURRENT_TIMESTAMP, FALSE)
                        """

# Query to insert a user into the database
# TODO change to parametrized executemany query
# [https://stackoverflow.com/questions/5616895/how-do-i-use-prepared-statements-for-inserting-multiple-records-in-sqlite-using]
INSERT_USER = """
                        INSERT OR IGNORE INTO users(id, name %s %s, addedToDB)
                        VALUES( "%s", "%s" %s %s, CURRENT_TIMESTAMP)
                    """

# Query to insert a commit into the database
INSERT_COMMIT = """
                          INSERT OR IGNORE INTO commits(id, repo, user, createdAt, additions, deletions, addedToDB)
                          VALUES("%s", "%s", "%s", %s, %s, %s, CURRENT_TIMESTAMP)
                      """

## Splitting up the Queries

Due to Githubs limitation on the number of 1000 items returned per query\[1\] we need to create queries which get less than 1000 items, but still cover the entirety of the dataset.

Previous attempts\[2\] to solve this exact problem constrained their queries by the amount of stars for each repository.
A method, which only works as long a there are less than 1000 repositories with the same amount of stars.

This was then mitigated by using the creation date of the repository as a second constraint.
As described in their corresponding blog article \[3\], this solution works by:

* First querying the Github Graphql API to see the result count of how many items a given query would provide
* If it is above a count of 1000 results the takes the date of jungest and oldest repository and splits the query in half of the time range
* Then the size of these two queries is checked again and if they are still above 1000 results the process is repeated until the size of the queries is below 1000 results

In [22]:
# Simple function to convert a Unix timestamp to a string in the format required by the github api
to_string = lambda stamp : datetime.fromtimestamp(stamp).strftime('%Y-%m-%dT%H:%M:%SZ')

In [ ]:
sections = {}

start = 1167609600 # Timestamp for 2007-01-01 (Github was founded in 2008 so this will cover all repos)
end = 1678209714  # Current Time stamp (for consistency will not use time.time()

amount_of_repos = None

# The query to get the number of repos in a given time range as well as the current state of the rate limit
count_query = ''' query { 
                   rateLimit {
                    cost
                    remaining
                    resetAt
                  }
                  search(
                    query:"is:public, stars:>%s, created:%s..%s"
                    type: REPOSITORY, first: 1) {
                    repositoryCount
                  }
                } '''


def split_querys(start, end):
  global amount_of_repos
  global repos_done

  repo_count_response = requests.post(
          'https://api.github.com/graphql',
          headers={'Authorization': 'bearer '+ config['API']['KEY']},
          json={"query": count_query % (config['GENERAL']['STARS'], to_string(start), to_string(end))}
      )
  
  # On the first run we get the total number of repos 
  # This is used to calculate the progress of the script
  if (amount_of_repos is None):
    amount_of_repos = repo_count_response.json()["data"]["search"]["repositoryCount"]
    sections["amount_of_repos"]=  amount_of_repos
    repos_done = 0

  # If we are close to the rate limit we sleep until the rate limit resets
  if repo_count_response.json()["data"]["rateLimit"]["remaining"] < 10:
    reset_time = datetime.strptime( repo_count_response.json()["data"]["rateLimit"]["resetAt"], '%Y-%m-%dT%H:%M:%SZ')
    
    while datetime.now() < reset_time:
      seconds_till_reset = (reset_time - datetime.now()).total_seconds()
      print ("Sleeping till %s... %d minutes and %d seconds left..." % ( reset_time, *divmod(seconds_till_reset, 60)))
      time.sleep(5)

  # If the number of repos in the repos in the time range is greater than 1000
  if repo_count_response.json()["data"]["search"]["repositoryCount"] > 1000:
    # We split the range in half and do the same query on each half
    # This will continue recursively until the number of repos is less than 1000
    split_querys(start, (start + end)//2)
    split_querys((start + end)//2, end) 
    
  else:
    # If we finnaly get a range with less than 1000 repos we add the timestamps to the sections list
    sections.update({f"{start}-{end}": {"start": start, "end": end, "inDb": False}})
    repos_done = repos_done+repo_count_response.json()["data"]["search"]["repositoryCount"]
    print(f"Working on {to_string(start)} to {to_string(end)}. Progress: {repos_done/amount_of_repos*100:.2f}% Status:{repo_count_response.status_code}")

split_querys(start, end)
with open('sections.json', 'w') as f:
    f.write(json.dumps(sections, indent=4))

### Adding the data to the database

While we are querying the API, we are also need a way to store the data we are getting.
It doen't make sense to store the entire data in memory to serialize it later on, because the size of the data is too big for that and if we where to run into an unresolvable error, we would lose all the data we have already queried.

This is done by adding the creator of every repository to the database and then adding the repository itself to the database with a reference to its creator.

We simply take the values straight from the provided json and add them to the database.
If an identical entry already exists, we don't add it again, but instead just use the existing one.

The data we recieve does not always contain the same information, so we need to check if the data is present and if not we add a null value to the database.

The values we expect from the API are:

todo: fill table

| Key | Type | Description |
| --- | --- | --- |
| id | String | The id of the repository |
....

For logging purposes we also add the date of the query to the database, so we can see how the data changes over time.


In [23]:
def add_repos_to_db(nodes):
    for node in nodes:
        item = node["node"]
        db.execute(INSERT_USER % (
                                ", createdAt" if item["owner"].get("createdAt") else "",
                                ", location" if item["owner"].get("location") else "",
                                item["owner"]["id"],
                                item["owner"].get("login") if item["owner"].get("login") else "",
                                f", \n {datetime.strptime(item['owner']['createdAt'], '%Y-%m-%dT%H:%M:%SZ').timestamp()}" if item["owner"].get("createdAt") else "",
                                # The following line is nasty but will do for now #TODO fix this
                                ", \n '%s'" % (item['owner']['location'].replace("'", r"''")) if item["owner"].get("location") else "",
                            )
                    )
        db.execute(INSERT_REPO % (
                                    f"primaryLanguage," if item.get("primaryLanguage") else "",
                                    item["id"],
                                    item["name"],
                                    item["owner"]["id"],
                                    item["url"],
                                    item["stargazerCount"],
                                    item["watchers"]["totalCount"],
                                    f'"{item["primaryLanguage"]["name"]}",' if item.get("primaryLanguage") else "",
                                    item["isFork"],
                                    item["forkCount"],
                                    datetime.strptime(item["updatedAt"], '%Y-%m-%dT%H:%M:%SZ').timestamp(),
                                    datetime.strptime(item["createdAt"], '%Y-%m-%dT%H:%M:%SZ').timestamp()
                                )
                    )
    db.commit()

## Querying the Github API

Now that we have can query the github with sizeable chunks of data, we can start to query the API.
We are still using the graphql API for this, as it enables us to fetch only the data we actually need.
The REST API would require us to fetch the entire repository object, which contains a lot of unnecessary and redundant data.

Just because we are now able to query the bite sized chunks of data, doesn't mean that the query will actually return them.
In order to keep the loading times of the website low, Github uses pagination to limit the amount of data returned per query.
This means that we can only get 100 items per query, which is why we need to use the `endCursor` to get the next 100 items.

The cursor functions like a little bookmark, which tells the API where we left off and where to continue from, it needs to be passed as a parameter to the next query.

### Querying the Repos

The query itself consist of 4 parts:

1. A little snippet, requesting the current state of the rate limit, so we can keep track of how many requests we have left and when to stop
2. The filter for the repositories consisting of the following:
    * Only repositories which are public (this is a bit redundant, as the API only returns public repositories or the ones you have access to)
    * A limit on the amount of stars the repository has, everything below 15 is being ignored as it indicates little relevance
    * The date range of the repositories, this is where we plug in our previously calculated date ranges
3. Then we request a little bit more metadata about the query itself, like the total count of items and the cursor for the next page and whether there is a next page at all
4. Then we tell the API exactly what kind of values we are interested in
   1. This being information about the repository itself, like the name, the url, the description, the creation date, the amount of stars and the amount of forks
   2. but also information about its creator, like the name, the profile creation date and its id.

In [24]:
repos_downloaded = 0 # Used to keep track of the progress of the script
repo_query= """
              {
                rateLimit {
                  cost
                  remaining
                  resetAt
                }
                search(
                  query: "is:public, stars:>%s, created:%s..%s"
                  %s
                  type: REPOSITORY
                  first: 100
                ) {
                  repositoryCount
                  pageInfo {
                    hasNextPage
                    endCursor
                  }
                  edges {
                    node {
                      ... on Repository {
                        createdAt
                        forkCount
                        isFork
                        updatedAt
                        primaryLanguage {
                          name
                        }
                        watchers {
                          totalCount
                        }
                        stargazerCount
                        databaseId
                        owner {
                          id
                          ... on User {
                            id
                            createdAt
                            location
                            databaseId
                            login
                          }
                        }
                        id
                        name
                        url
                      }
                    }
                  }
                }
              }"""



with open('sections.json') as json_file:
  file = json.load(json_file)
  amount_of_repos = file["amount_of_repos"] 
  sections = file["sections"]


for index in [x for x in sections if sections[x]["inDb"] == False]:
  section = sections[index]

  print("=" * 100)
  print(f"Downloading repos from {to_string(section['start'])} to {to_string(section['end'])}")

  
  cursor = None # Used to keep track of the current page in the query
  has_next_page = True # Used to indicate if there are more pages to query

  
  while (has_next_page):
    print("-"*100)
    
    repo_query_response = requests.post(
              'https://api.github.com/graphql',
              headers={'Authorization': 'bearer '+ config['API']['KEY']},
              json={"query": repo_query % (
                config['GENERAL']["STARS"],
                to_string(section["start"]),
                to_string(section["end"]),
                f"after: \"{cursor}\"" if cursor else ""
              )}
          )
    # If we are close to the rate limit we sleep until the rate limit resets
    if repo_query_response.json()["data"]["rateLimit"]["remaining"] < 10:
      reset_time = datetime.strptime( repo_query_response.json()["data"]["rateLimit"]["resetAt"], '%Y-%m-%dT%H:%M:%SZ')
      while datetime.now() < reset_time:
        seconds_till_reset = (reset_time - datetime.now()).total_seconds()
        print ("Sleeping till %s... %d minutes and %d seconds left..." % ( reset_time, *divmod(seconds_till_reset, 60)))
        time.sleep(5)

    # Summing up the progress made so far
    repos_downloaded = repos_downloaded + len(repo_query_response.json()["data"]["search"]["edges"])
    
    # Updating the cursor and has_next_page variables to know if and where to continue the query
    cursor = repo_query_response.json()["data"]["search"]["pageInfo"]["endCursor"]
    has_next_page = repo_query_response.json()["data"]["search"]["pageInfo"]["hasNextPage"]
    
    # Adding the repos to the nodes list
   
    add_repos_to_db(repo_query_response.json()["data"]["search"]["edges"])
    section["inDb"] = True
    
    file["sections"] = sections
    with open('sections.json', 'w+') as outfile:
      outfile.write(json.dumps(file, indent=4))

    # Presenting the progress of the script
    print(f"""Downloaded {repos_downloaded}/{amount_of_repos} repos \
          Requests left: {repo_query_response.json()['data']['rateLimit']['remaining']} \
          Progress: {repos_downloaded/amount_of_repos*100:.2f}%.""")
  
  


----------------------------------------------------------------------------------------------------

                        INSERT OR IGNORE INTO users(id, name , createdAt , location, addedToDB)
                        VALUES( "MDQ6VXNlcjE=", "mojombo" , 
 1192857859.0 , 
 'San Francisco', CURRENT_TIMESTAMP)
                    
Downloaded 1/1 repos           Requests left: 4956           Progress: 100.00%.
----------------------------------------------------------------------------------------------------

                        INSERT OR IGNORE INTO users(id, name  , addedToDB)
                        VALUES( "MDEyOk9yZ2FuaXphdGlvbjU2Nzg1MzU2", ""  , CURRENT_TIMESTAMP)
                    

                        INSERT OR IGNORE INTO users(id, name  , addedToDB)
                        VALUES( "MDEyOk9yZ2FuaXphdGlvbjU2NzAy", ""  , CURRENT_TIMESTAMP)
                    

                        INSERT OR IGNORE INTO users(id, name  , addedToDB)
                        VALUES( "

KeyboardInterrupt: 

### Queries for the Commits

But we do not only want the data about the repositories and the person who inially created them, we also want to know who actually worked on them.
This is where the commits come in...

The commits are the actual changes to the code, which are made by contributors. 
We could have queried this during the repository query, but this seems to overload the API and also would make the code utterly unreadable.

This is why we simply query the commits for every repository separately, which makes the query itself a lot simpler.

To get to the commits we need to use the creator name and the repository name, as the api does not accept the repository id itself.
Unfortumately the creator name is not always provided by our query, this is due to the fact, that not only users can create repositories, but also organizations.
If a repository is created by an organization, the creator name is not provided by the API call directly.

Indirectly we can get the creator name by spitting up the repository name by the `/` and taking the first part of the string, which is the name of the organization.
Whith this information we can then query the API for the commits.

The structually the querry is very similar to the one for the repositories, we are still using the graphql API and we are still traversing 

In [ ]:
df = pd.read_sql_query("""
                       SELECT users.id, repos.url
                       FROM repos
                       JOIN users ON repos.owner = users.id
                       WHERE repos.allCommits is False
                       """, db)
repos = df["url"].str.split("/").str[3:5].values.tolist()
print(repos.head())

In [ ]:
for repo in repos:
    cursor = None # Used to keep track of the current page in the query
    has_next_page = True # Used to indicate if there are more pages to query
    commits_total = None # Used to keep track of the total number of commits
    commits_downloaded = 0
    commit_query = """
                     {
                      rateLimit {
                        cost
                        remaining
                        resetAt
                      }
                      repository(name: "%s", owner: "%s") {
                        id,
                        defaultBranchRef {
                          target {
                            ... on Commit {
                              id
                              history(first: 100 %s) {
                                edges {
                                  node {
                                    id
                                    committedDate
                                    additions
                                    deletions   
                                    author {
                                      user {
                                        id
                                        login
                                        location
                                      }
                                    }
                                  }
                                }
                                totalCount
                                pageInfo {
                                  endCursor
                                  hasNextPage
                                }
                              }
                            }
                          }
                        }
                      }
                    }"""

    while (has_next_page):
        commit_query_response = requests.post(
                  'https://api.github.com/graphql',
                  headers={'Authorization': 'bearer '+ config['API']['KEY']},
                  json={"query": commit_query % (repo[1], repo[0] , f"after: \"{cursor}\"" if cursor else "" )}
              )
        
        # If we encounter an error we write it to the error log
        if commit_query_response.status_code != 200:
            with open("error_log.txt", "a+") as error_log:
                error_log.write(f"{repo[0]}/{repo[1]}: {commit_query_response}")
        
        if commit_query_response.json()["data"]["rateLimit"]["remaining"] <= 10:
            reset_time = datetime.strptime(commit_query_response.json()["data"]["rateLimit"]["resetAt"], '%Y-%m-%dT%H:%M:%SZ')
            while datetime.now() < reset_time:
                seconds_till_reset = (reset_time - datetime.now()).total_seconds()
                print ("Sleeping till %s... %d minutes and %d seconds left..." % ( reset_time, *divmod(seconds_till_reset, 60)))
                time.sleep(5)

        
        if commits_total == None:
            commits_total = commit_query_response.json()["data"]["repository"]["defaultBranchRef"]["target"]["history"]["totalCount"]    
        commits_downloaded = commits_downloaded + len(commit_query_response.json()["data"]["repository"]["defaultBranchRef"]["target"]["history"]["edges"])

        cursor = commit_query_response.json()["data"]["repository"]["defaultBranchRef"]["target"]["history"]["pageInfo"]["endCursor"]
        has_next_page = commit_query_response.json()["data"]["repository"]["defaultBranchRef"]["target"]["history"]["pageInfo"]["hasNextPage"]
                
        for commit in commit_query_response.json()["data"]["repository"]["defaultBranchRef"]["target"]["history"]["edges"]:
            commit = commit["node"]

            
            if commit.get("author") and commit["author"].get("user"): 
              db.execute(INSERT_USER % (
                              ", createdAt" if commit["author"]["user"].get("createdAt") else "",
                              ", location" if commit["author"]["user"].get("location") else "",
                              commit["author"]["user"]["id"],
                              commit["author"]["user"].get("login") if commit["author"]["user"].get("login") else "",
                              f", \n {datetime.strptime(commit['author']['user']['createdAt'], '%Y-%m-%dT%H:%M:%SZ').timestamp()}" if commit["author"]['user'].get("createdAt") else "",
                              f", \n \"{commit['author']['user']['location']}\"" if commit["author"]['user'].get("location") else "",
                          )
                      )

            db.execute(INSERT_COMMIT % (
                                commit["id"],
                                commit_query_response.json()["data"]["repository"]["id"],
                                commit["author"]["user"]["id"] if commit.get("author") and commit["author"].get("user") else "",
                                datetime.strptime(commit["committedDate"], '%Y-%m-%dT%H:%M:%SZ').timestamp(),
                                commit["additions"],
                                commit["deletions"]
                            )
            )
        print(f"Progress: {commits_downloaded/commits_total*100:.2f}% \t  Requests left: {commit_query_response.json()['data']['rateLimit']['remaining']} \t Current Repo: {repo[1]} ")

    db.execute(f""" UPDATE repos
                    SET allCommits = True
                    WHERE id = "{
                      commit_query_response.json()["data"]["repository"]["id"]
                      }"
                """)
    db.commit()


Progress: 19.38% 	  Requests left: 4517 	 Current Repo: grit 
Progress: 38.76% 	  Requests left: 4516 	 Current Repo: grit 
Progress: 58.14% 	  Requests left: 4515 	 Current Repo: grit 
Progress: 77.52% 	  Requests left: 4514 	 Current Repo: grit 
Progress: 96.90% 	  Requests left: 4513 	 Current Repo: grit 
Progress: 100.00% 	  Requests left: 4512 	 Current Repo: grit 
Progress: 7.59% 	  Requests left: 4511 	 Current Repo: nerdtree 


KeyboardInterrupt: 

## Bibliography

[1] “Resources in the REST API,” GitHub Docs. https://docs.github.com/en/rest/overview/resources-in-the-rest-api?apiVersion=2022-11-28 (accessed Mar. 07, 2023).

[2] danvk, “How can I get a list of all public GitHub repos with more than 20 stars?,” Stack Overflow, Feb. 02, 2020. https://stackoverflow.com/q/60022429 (accessed Mar. 07, 2023).


[3] D. Vanderkam, “GitHub Stars and the h-index: A Journey,” Medium, Feb. 10, 2020. https://danvdk.medium.com/github-stars-and-the-h-index-a-journey-c104cfe37da6 (accessed Mar. 06, 2023).